### Creating a class for batching the data

Tensorflow has appropriate methods for batching data but we will create a custom method

In [1]:
import numpy as np

# Create a class that will do the batching for the algorithm
class Audiobooks_Data_Reader():
    #Dataset is a mandatory argument, while the batch_size is optional
    #if you don't input batch_size, it will automatically take the value: None
    def __init__(self, dataset, batch_size=None):
        npz = np.load(dataset)
        
        #two variables that take the values of the inputs and targets. Inputs are floats, targets are integers
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # counts the batch number, given the size you feed it later
        #if the batch size is None, we are either validating or testing, so we want to take the data in a single batch
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
        
    # A method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        #you slice the dataset in batches and then the "next" function loads them one after the other
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        #One-hot encode the targets. In this exampmle it is a bit superfluous since we have a 0/1 column
        # as a target already but we're giving you the code regardless, as it will be useful for any
        # classification task with more than one target column
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        #the function will return the inputs batch and the one-hot encoded targets
        return inputs_batch, targets_one_hot
    
    # A method needed for iterating over the batches, as we will put them in a loop
    # This tells Python that the class we're defining is iterable, i.e. that we can use it like:
    # for input, output in data:
    # do things
    # An iterator in python is a class with a method __next__ that defines exactly how to iterate through its objects
    def __iter__(self):
        return self

### Creating the Machine learning algorithm

In [2]:
#we need to import TF, but we have updated to TF v2.0 so we will be using some deprecated code hence we will revert to v1 
import tensorflow.compat.v1 as tf
#from tensorflow.python.framework import ops


tf.disable_v2_behavior()
#input size depends on the number of input variables
input_size = 10
# Output size is 2, as we one_hot encoded the targets
output_size = 2
# choose a hidden_layer size
hidden_layer_size = 50

# Reset the default graph, so you can fiddle with the hyperparameters and then rerun the code.
tf.reset_default_graph()

# Create the placeholders
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.int32, [None, output_size])

# Outline the model. We will create a net with 2 hidden layers
weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])
#we will incorporate the softmax activation into the loss
outputs = tf.matmul(outputs_2, weights_3) + biases_3



Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
# Use the softmax cross entropy loss with logits
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
mean_loss = tf.reduce_mean(loss)

#get a 0 or 1 for every input indicating whether it output the correct answer
out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

#Optimize with Adam
optimize = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(mean_loss)

#create session
sess = tf.InteractiveSession()

#initialize the variables
initializer = tf.global_variables_initializer()
sess.run(initializer)

#choose the batch size
batch_size = 100

#early stopping mechanisms
max_epochs = 100
prev_validation_loss = 9999999.


In [11]:
#Load the first batch of training and validation, using the data reader class
train_data = Audiobooks_Data_Reader('Audiobooks_training_data.npz', batch_size)
validation_data = Audiobooks_Data_Reader('Audiobooks_validation_data.npz')

C:\Users\HP OMEN\AppData\Local\Temp\ipykernel_30364\917757533.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
C:\Users\HP OMEN\AppData\Local\Temp\ipykernel_30364\917757533.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; 

In [18]:
#creating loops for epochs
for epoch_counter in range(max_epochs):
    
    #set the epoch loss to 0, and make it a float
    curr_epoch_loss = 0.
    
    #iterate over the training data
    #since train_data is an instance of Audiobooks_Data_Reader class,
    #we can iterate through it by implicitly using the __next__ method we defined above
    #it batches samples together, one-hote encodes the targets, and returns inputs and targets batch by batch
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss],
                                feed_dict={inputs: input_batch, targets: target_batch})
        
        #record the batch loss into the current epoch loss
        curr_epoch_loss += batch_loss
        
    # find the mean curr_epoch loss
    # batch_count is a variable, definded in the Audiobooks_Data_reader class
    curr_epoch_loss /= train_data.batch_count
    
    # set validation loss and accuracy for the epoch to zero
    validation_loss = 0.
    validation_accuracy = 0.
    
    # Use the same logic of the code to forward propagate the validation set
    # there will be a single batch, as the class was created in this way
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
                                                       feed_dict={inputs: input_batch, targets: target_batch})
        
        # Print statistics for the current epoch
        print('Epoch '+str(epoch_counter+1) + 
              '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
              '. validation loss: '+'{0:.3f}'.format(validation_loss)+
              '. validation accuracy: '+'{0:.3f}'.format(validation_accuracy * 100)+'%')
        
        #trigger early stopping if validation loss begins increasing.
        if validation_loss > prev_validation_loss:
            break

Epoch 2. Training loss: 0.626. validation loss: 0.614. validation accuracy: 68.009%
Epoch 3. Training loss: 0.609. validation loss: 0.596. validation accuracy: 71.141%
Epoch 4. Training loss: 0.593. validation loss: 0.578. validation accuracy: 74.273%
Epoch 5. Training loss: 0.577. validation loss: 0.561. validation accuracy: 78.076%
Epoch 6. Training loss: 0.562. validation loss: 0.546. validation accuracy: 78.076%
Epoch 7. Training loss: 0.548. validation loss: 0.530. validation accuracy: 78.523%
Epoch 8. Training loss: 0.534. validation loss: 0.516. validation accuracy: 79.195%
Epoch 9. Training loss: 0.521. validation loss: 0.502. validation accuracy: 78.971%
Epoch 10. Training loss: 0.509. validation loss: 0.489. validation accuracy: 79.195%
Epoch 11. Training loss: 0.497. validation loss: 0.477. validation accuracy: 79.195%
Epoch 12. Training loss: 0.486. validation loss: 0.466. validation accuracy: 79.642%
Epoch 13. Training loss: 0.475. validation loss: 0.455. validation accura

Epoch 100. Training loss: 0.336. validation loss: 0.313. validation accuracy: 82.103%
